In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import cv2, os, pandas as pd

This code reads the slice data for the inference files, and labels the score according to what the model recognised that section as. 
Therefore, it allows a quick visual check on what features were being recognised by the model to try and evaluate weaknesses in the training data. 

In [2]:
##Navigate to the directory, and define the files of that contain the scores for each slice
os.chdir('E:/5.Machine Learning/7.Unlabelled Data Test/ML Categorised/Quality Cut-off 83.125/')

image_locs='E:/5.Machine Learning/7.Unlabelled Data Test/ML Categorised/Quality Cut-off 83.125/'

segment_txtfiles=['Unlabelled_data_repeat-S2-A03__results_02.txt','Unlabelled_data_repeat-S3-A01__results_02.txt']
segment_px_dimensions=250

In [3]:
# Check the crop dimensions are correct. 
image = cv2.imread('0.0_2021-01-18 - 11.04.16 Frame 012 Trace 2450um along_pixelsquare.jpg')
shape=image.shape
crop_img = image[56:1056,65:1065]
cv2.imshow('hello',crop_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [29]:
print(shape[1])

1256


In [4]:
def get_old_imagename(image_name):
    if image_name[0]=='/':
        image_name=image_name[1:]     #Gets rid of the "/" at the start of the filename for the norm datasets.
    image_name=image_name.replace('_',' ')    #Undoes the underscoring of names done for AWS, to match the image names in folder
    
    image_name=image_name.rsplit(' ', 2)[0]     #removes the "1000 pixel square" to be replaced with ones with underscores
    
    original_imagename=image_name+' along_pixelsquare.jpg'    #creates the filename ending as per the original images
    
    return original_imagename

In [5]:
def overlay_transparent(background, overlay, x, y, alpha):
    #https://stackoverflow.com/questions/40895785/using-opencv-to-overlay-transparent-image-onto-another-image
    
    background_width = background.shape[1]
    background_height = background.shape[0]
    
    #If coordinates given go off the image coordinates
    if x >= background_width or y >= background_height:
        print('coordinates exceeded limits for',x,y)
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        #Crops any region of the width of the overlay that will go past background coords
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        #Crops any region of the height of the overlay that will go past background coords
        overlay = overlay[:h]
    
    #Figure out what this bit means... something to do with colour.
    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [overlay,np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255],
            axis = 2)

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0
    
    #my bodgeline below. 
    mask=alpha
    background[y:y+h, x:x+w] = (1 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    return background

In [6]:
# Main code to label the image files from inference. Check the location. 
#old_image_loc=""
os.chdir('E:/5.Machine Learning/7.Unlabelled Data Test/') # Location of the segment_txtfiles results
ful_results=['Unlabelled_data_repeat-S2-A03__full_results_02.txt', 'Unlabelled_data_repeat-S3-A01__full_results_02.txt']
imagequal_data1 = pd.read_csv(ful_results[0], sep=" ") 
imagequal_data2 = pd.read_csv(ful_results[1], sep=" ") 
imagequal = [imagequal_data1, imagequal_data2]

imagequaldata = pd.concat(imagequal, ignore_index=True)

for txtfile in segment_txtfiles:
    os.chdir('E:/5.Machine Learning/7.Unlabelled Data Test/') # Location of the segment_txtfiles results
    data = pd.read_csv(txtfile, sep=" ") # Read the data as a pandas dataframe
    new_file_row=data.iloc[::16, :]   # Sets the limit to define a new file. For 4x4 slices, this would be 16. 
    counter=0
    for image_index, image_row in new_file_row.iterrows():
        imagename = image_row['Filename'] #Gets the filename from the txt file
        original_imagename = get_old_imagename(imagename) #Finds the image filename using the pre-defined function above.
        #print(original_imagename)
        imagequality_score=9999999
        #Find the image quality score for the associated frame
        for index, row in imagequaldata.iterrows():  
            if row['Filename']==imagename:
                imagequality_score=row['Rekognition']
                
        if imagequality_score==9999999:
            print('Image quality not found for',imagename)
            break

        imagequality_name=str(imagequality_score)+'_'+original_imagename

        #Find the image in the categories inference image directory folders (walking between FP/FN/TP/TN)
        for root, dirs, files in os.walk(image_locs):
            if imagequality_name in files:
                #print("root = ",root)
                old_image_loc= os.path.join(root, imagequality_name)
                old_image_loc=old_image_loc.replace("\\","/")

        oldpath=os.path.dirname(old_image_loc)
        os.chdir(oldpath)

        # load the image
        image = cv2.imread(imagequality_name) 
        #height, width, channels = image.shape
        #print (height, width, channels)          #1064 1248 3
        #crop_img = image[60:1060, 68:1068]       #Y coords and X coords of the AFM image. 
        
        #Identify the 16 segments' data for this image
        slice_rows=data[image_index : image_index+16]
        
        # create a copy of the original image -- for the final output image
        output = image.copy()
        # create overlay copy of the original image -- one for the overlay and one for the final output image
        overlay = image.copy()
        #txtoverlay = image.copy()
        
        #For each of the 16 slices:
        for slice_index, slice_row in slice_rows.iterrows():  
            rekog_anom_score =  slice_row['Rekognition_anom']
            #print(type(rekog_anom_score))
            if rekog_anom_score>0.1:
                segment_no= slice_row['Section']
                ##Check the values at start to ensure they correspond to the starting pixel crop locations.
                y_coord=56+(int(segment_no[0])*segment_px_dimensions)
                x_coord=65+(int(segment_no[2])*segment_px_dimensions)

                y_end_coord=56+((int(segment_no[0])+1)*segment_px_dimensions)
                x_end_coord=65+((int(segment_no[2])+1)*segment_px_dimensions)

                # draw a rectangle in the image
                # represents the top left corner of rectangle
                start_point = (x_coord+1, y_coord+1)
                # represents the bottom right corner of rectangle
                end_point = (x_end_coord-1, y_end_coord-1)

                #Create an overlay for that segment
                if slice_row['Label']=='anomaly':
                    color = (0, 0, 255)      # Red color in BGR
                    alpha=rekog_anom_score
                else:
                    color = (0, 215, 255)    # Orange
                    alpha=rekog_anom_score+0.3
                    
                #cv2.rectangle(overlay, start_point, end_point, color, -1) #-1 fills rectangle
                cv2.rectangle(overlay, start_point, end_point, color, 2) #border of rectangle only
                cv2.putText(overlay, str(rekog_anom_score), (start_point[0]+75, start_point[1]+125), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)
                

                # apply the overlay
                #output=cv2.addWeighted(overlay, alpha, output, 1-alpha, 0, output)
                crop_overlay = overlay[start_point[1]:end_point[1]+1, start_point[0]:end_point[0]+1]
                output=overlay_transparent(output, crop_overlay, start_point[0], start_point[1], alpha)   

        
        # show the output image
        #small_output = cv2.resize(output, (int(1248 *0.75), int(1064*0.75)))
        #cv2.imshow(str(counter)+' - '+original_imagename[:-4], small_output)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        
        counter+=1
        if str(counter).endswith('0'):
            print(counter)
#         if counter==5:
#            break
        
        # Saving the image
        labelled_filename= str(imagequality_score)+'_'+original_imagename[:-4]+'_rek-score1.jpg'
        cv2.imwrite(labelled_filename, output)
    #break   ##Delete this line when doing both files!!!!!!!
            


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710


In [134]:
#os.chdir('E:/6.Images for ML Inference/Inference Results - 1000x1000 inference7/ML Categorised/')
2021-05-11 - 10.35.50 Frame 004 Trace 25um along_1000_pixelsquare
# load the image
#image = cv2.imread("2021-05-11 - 10.35.50 Frame 036 Trace 225um along_1000_pixelsquare.jpg")

#height, width, channels = image.shape
#print (height, width, channels)

#crop_img = image[60:1060, 68:1068] #X coords and Y coords of the AFM image. 
#cv2.imshow("cropped", crop_img)
#cv2.waitKey(0)

1064 1248 3


In [36]:
# loop over the alpha transparency values
#for alpha in np.arange(0, 1.05, 0.05)[::-1]:
    
    # create two copies of the original image -- one for the overlay and one for the final output image
  #  overlay = image.copy()
  #  output = image.copy()
    
    # draw a red rectangle in the image
    # represents the top left corner of rectangle
 #   start_point = (5, 5)
    # represents the bottom right corner of rectangle
  #  end_point = (220, 220)
    
    # Red color in BGR
 #   color = (0, 0, 255)
    #cv2.rectangle(overlay, start_point, end_point, color, -1) #-1 fills rectangle
 #   cv2.rectangle(overlay, start_point, end_point, color, 3) #border of rectangle only

    #cv2.putText(overlay, "PyImageSearch: alpha={}".format(alpha), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
    
    # apply the overlay
  #  cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)
    
    # show the output image
    #print("alpha={}, beta={}".format(alpha, 1 - alpha))
   # cv2.imshow("Output"+str(alpha), output)
    #cv2.waitKey(0)
   # cv2.destroyAllWindows()

In [ ]:

# Saving the image
#cv2.imwrite(filename, image)